In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('events.csv', low_memory = False)

In [14]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-31 23:38:05,ad campaign hit,0004b0a2,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-31 23:38:05,visited site,0004b0a2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Paid,New,Camaragibe,Pernambuco,Brazil,Smartphone,360x640,Android 6,Chrome Mobile 39
2,2018-05-31 23:38:09,viewed product,0004b0a2,NaN,2694.0,iPhone 5s,Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-31 23:38:40,checkout,0004b0a2,NaN,2694.0,iPhone 5s,Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-29 13:29:25,viewed product,0006a21a,NaN,15338.0,Samsung Galaxy S8,Bom,64GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Imprimo el uso inicial de memoria y los tipos de las columnas

In [15]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011288 entries, 0 to 1011287
Data columns (total 23 columns):
timestamp                   1011288 non-null object
event                       1011288 non-null object
person                      1011288 non-null object
url                         82756 non-null object
sku                         563838 non-null object
model                       564284 non-null object
condition                   563836 non-null object
storage                     563836 non-null object
color                       563836 non-null object
skus                        221699 non-null object
search_term                 48967 non-null object
staticpage                  3598 non-null object
campaign_source             82796 non-null object
search_engine               50957 non-null object
channel                     87378 non-null object
new_vs_returning            87378 non-null object
city                        87378 non-null object
region                    

### Veo a que columnas les puedo cambiar el tipo

In [17]:
df.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,1011288,1011288,1011288,82756,563838,564284,563836,563836,563836,221699,...,50957,87378,87378,87378,87378,87378,87378,87378,87378,87378
unique,793805,11,27624,227,3574,202,5,8,63,35310,...,4,7,2,1939,93,46,4,282,121,343
top,2018-05-15 15:56:06,viewed product,71492f2b,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,11,528931,2771,28323,4282,50916,243014,190833,132960,594,...,50240,44193,60480,15819,24996,84308,44239,30009,19675,22611


### Cambio el tipo de 'person' a uint32 y 'timestamp' a datetime64

In [18]:
df['person'] = df['person'].apply(lambda x: int(x, 16)).astype('uint32')
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [19]:
df['search_term'] = df['search_term'].str.lower()

### Convierto a categorías el resto de las columnas

In [20]:
for col in df.columns:
    num_unique_values = len(df[col].unique())
    num_total_values = len(df[col])
    if df[col].dtype == np.object and num_unique_values * 1.0 / num_total_values < 0.5:
        df[col] = df[col].astype('category')

### Imprimo el uso de memoria resultante

In [21]:
df.info (memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011288 entries, 0 to 1011287
Data columns (total 23 columns):
timestamp                   1011288 non-null datetime64[ns]
event                       1011288 non-null category
person                      1011288 non-null uint32
url                         82756 non-null category
sku                         563838 non-null category
model                       564284 non-null category
condition                   563836 non-null category
storage                     563836 non-null category
color                       563836 non-null category
skus                        221699 non-null category
search_term                 48967 non-null category
staticpage                  3598 non-null category
campaign_source             82796 non-null category
search_engine               50957 non-null category
channel                     87378 non-null category
new_vs_returning            87378 non-null category
city                        87378 non-nu

In [22]:
df['search_term'].cat.categories

Index([u'"galaxy tab e 9.6 wi-fi"', u'"samsung galaxy j1"', u'#2793522',
       u',moto z.', u'.', u'. galaxy j7 prime', u'1 carregador alternativo',
       u'100', u'1020', u'128',
       ...
       u'~s8', u'Ásia', u'Ásia zenfone 4', u'Ásus zenfone 5',
       u'Ásus zenfone 5z', u'Ícone', u'Ícone 5', u'Ícone 7', u'Ífone 5',
       u'ĺpl7u(não   wqenhghiulsaq'],
      dtype='object', length=4293)

In [23]:
for cat in df['search_term'].cat.categories:
    print('cat: %s'%(cat))

cat: "galaxy tab e 9.6 wi-fi"
cat: "samsung galaxy j1"
cat: #2793522
cat: ,moto z.
cat: .
cat: . galaxy j7 prime
cat: 1 carregador alternativo
cat: 100
cat: 1020
cat: 128
cat: 128bgigas
cat: 128gb
cat: 128mb
cat: 16 g
cat: 16 gigas tela 5
cat: 16g
cat: 16gb
cat: 16gb 4g dual tela5"
cat: 18-03737
cat: 2 gb
cat: 2 x power
cat: 200
cat: 200,00
cat: 2017
cat: 21 mp
cat: 2624858
cat: 278
cat: 2a via boleto
cat: 2gb ram
cat: 3 gb de ram
cat: 3 ram
cat: 300
cat: 300 a 400 reais
cat: 300 reais
cat: 300,0
cat: 300,00
cat: 32 gb
cat: 32 giga
cat: 32g
cat: 32gb
cat: 3gb
cat: 3gb ram
cat: 4 gb ram
cat: 4 mini
cat: 4 s
cat: 4c
cat: 4g
cat: 4g 2 gb ram
cat: 4s
cat: 5
cat: 5 c
cat: 5 iphone
cat: 5 prime
cat: 5,5
cat: 50
cat: 5904
cat: 5920
cat: 5c
cat: 5c azul
cat: 5c branco 32gb
cat: 5c branco 32gb excelente
cat: 5s
cat: 5s 16gb
cat: 5s 32 gb
cat: 5s 32gb
cat: 5s cinza
cat: 5s dourado
cat: 5s dourado 32gb
cat: 5s gold
cat: 5s rosa
cat: 5se
cat: 6
cat: 6 16gb cinza
cat: 6 64gb
cat: 6 galaxy
cat: 6 ip

cat: galaxy s8 128gb
cat: galaxy s8 64gb
cat: galaxy s8 ametista
cat: galaxy s8 edeg
cat: galaxy s8 plus
cat: galaxy s8+
cat: galaxy s85
cat: galaxy s9
cat: galaxy s9 splus
cat: galaxy samsung granprime
cat: galaxy tab
cat: galaxy tab 9,6
cat: galaxy tab e
cat: galaxy tab e 9.6
cat: galaxy tab e 9.6 wi-fi
cat: galaxy tanto
cat: galaxy win
cat: galaxy win 2
cat: galaxy win 3
cat: galaxy win-2 g360
cat: galaxy win-duos
cat: galaxy win-duos g360
cat: galaxy win2
cat: galaxy. s4
cat: galaxy. s4 9015
cat: galaxy8
cat: galaxya7
cat: galaxys j5
cat: galaxys5 mini
cat: galaxyss7
cat: galay a7 2017
cat: galay core
cat: galazy s7 edge
cat: galex j7 prime
cat: galexy j7 prime
cat: gallaxy s5
cat: galxy s7
cat: galxy s8
cat: galzxy s7 edge
cat: galáctico j 7 prime
cat: galáctico s7
cat: galáctico s7 edge
cat: galáctico s8
cat: galáx s6
cat: galáx s6 edge
cat: galáxi s6 edej
cat: galáxia a5 2017
cat: galáxia j
cat: galáxia j5
cat: galáxia j7
cat: galáxia s6
cat: galáxia s7 edge
cat: galáxias j5
cat

cat: moto e4plus
cat: moto e5
cat: moto e7
cat: moto ee
cat: moto f 2
cat: moto g
cat: moto g 1
cat: moto g 2
cat: moto g 2 geração
cat: moto g 2ª geração
cat: moto g 3
cat: moto g 3 geração
cat: moto g 32gb
cat: moto g 3moto
cat: moto g 4
cat: moto g 4 e
cat: moto g 4 pl
cat: moto g 4 play
cat: moto g 4 plus
cat: moto g 4 plus branco
cat: moto g 4 pus
cat: moto g 4 s plus
cat: moto g 4 usado
cat: moto g 4plus
cat: moto g 5
cat: moto g 5  plus
cat: moto g 5 plkus
cat: moto g 5 plus
cat: moto g 5 s
cat: moto g 5 s plus
cat: moto g 5]
cat: moto g 5plus
cat: moto g 5pus
cat: moto g 5s
cat: moto g 5s plus
cat: moto g 6
cat: moto g 6 plus
cat: moto g 7
cat: moto g 8
cat: moto g colors
cat: moto g dois
cat: moto g e 4
cat: moto g e4
cat: moto g foce
cat: moto g g4 o
cat: moto g g4 plus
cat: moto g g5 plus
cat: moto g g6 plus
cat: moto g play
cat: moto g plus
cat: moto g plus motorola
cat: moto g x
cat: moto g x4
cat: moto g1
cat: moto g1 geraçao
cat: moto g2
cat: moto g2 16 gb
cat: moto g2 1

cat: sansumg on 7
cat: sansumg prime
cat: sansumg s6 flat
cat: sansumg s7
cat: sansumgue
cat: sansun g j7
cat: sansung
cat: sansung 32g
cat: sansung 6
cat: sansung 6 s
cat: sansung a
cat: sansung a 2017
cat: sansung a 5
cat: sansung a3
cat: sansung a3 2016
cat: sansung a5
cat: sansung a5 2017
cat: sansung a7
cat: sansung a7 2016
cat: sansung a8
cat: sansung a8 2018
cat: sansung a9
cat: sansung com tv
cat: sansung edge
cat: sansung galax 7s
cat: sansung galax a8
cat: sansung galax j7
cat: sansung galax j7 prime
cat: sansung galaxi j7 prime
cat: sansung galaxi s7 edge
cat: sansung galaxy
cat: sansung galaxy  j2
cat: sansung galaxy  j7
cat: sansung galaxy  j7 prime
cat: sansung galaxy a9
cat: sansung galaxy j
cat: sansung galaxy j1
cat: sansung galaxy j1 minie
cat: sansung galaxy j2
cat: sansung galaxy j5
cat: sansung galaxy j5 prime
cat: sansung galaxy j5 pro 32 gb
cat: sansung galaxy j7
cat: sansung galaxy s 7 32 preto na categoria
cat: sansung galaxy s 7 32 preto na categoria exc
cat: 